**the wisdom of crowds**  
**social proof**

In [1]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.width',500)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('display.float_format',lambda x:'%.5f' % x)

In [2]:
df=pd.read_csv('/content/course_reviews.csv')
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000


In [3]:
df.shape

(4323, 6)

In [4]:
#rating dagılımı
df['Rating'].value_counts()

,count
Rating,
5.00000,3267
4.50000,475
4.00000,383
3.50000,96
3.00000,62
1.00000,15
2.00000,12
2.50000,11
1.50000,2


In [5]:
df['Questions Asked'].value_counts()

,count
Questions Asked,
0.00000,3867
1.00000,276
2.00000,80
3.00000,43
4.00000,15
5.00000,13
6.00000,9
8.00000,5
9.00000,3


In [6]:
df.groupby('Questions Asked').agg({'Questions Asked':'count',
                                   'Rating':'mean'})

,Questions Asked,Rating
Questions Asked,,
0.00000,3867,4.76519
1.00000,276,4.74094
2.00000,80,4.80625
3.00000,43,4.74419
4.00000,15,4.83333
5.00000,13,4.65385
6.00000,9,5.00000
7.00000,2,4.75000
8.00000,5,4.90000


In [7]:
# ortalama puan
df['Rating'].mean()

np.float64(4.764284061993986)

**Time - Based Weighted Average**

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rating              4323 non-null   float64
 1   Timestamp           4323 non-null   object 
 2   Enrolled            4323 non-null   object 
 3   Progress            4323 non-null   float64
 4   Questions Asked     4323 non-null   float64
 5   Questions Answered  4323 non-null   float64
dtypes: float64(4), object(2)
memory usage: 202.8+ KB


In [9]:
df['Timestamp']=pd.to_datetime(df['Timestamp'])

In [10]:
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000


In [11]:
current_date=pd.to_datetime('2021-02-11 0:0:0')

In [12]:
df['days']=(current_date-df['Timestamp']).dt.days

In [13]:
# son 30 gun icindeki yorumlar
df[df['days']<=30]

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,days
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000,5
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000,6
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000,6
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000,6
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000,6
5,4.00000,2021-02-04 12:42:36,2021-02-01 15:40:13,1.00000,0.00000,0.00000,6
6,5.00000,2021-02-04 12:25:30,2020-11-30 19:23:54,85.00000,0.00000,4.00000,6
7,4.50000,2021-02-04 11:13:15,2021-01-08 08:05:42,10.00000,0.00000,0.00000,6
8,5.00000,2021-02-04 08:59:53,2021-02-02 18:14:49,1.00000,0.00000,0.00000,6
9,5.00000,2021-02-03 22:40:04,2020-08-01 22:30:42,35.00000,0.00000,0.00000,7


In [14]:
# son 30 gun icindeki yorum sayisi
df[df['days']<=30].count()

,0
Rating,192
Timestamp,192
Enrolled,192
Progress,192
Questions Asked,192
Questions Answered,192
days,192


In [15]:
# rating ortalamasi
df.loc[df['days']<=30,'Rating'].mean()

np.float64(4.7734375)

In [16]:
# 30'dan buyuk, 90'dan kucuk degerlerin rating ortalamasi
df.loc[(df['days']>30) & (df['days']<=90),'Rating'].mean()

np.float64(4.76195219123506)

In [17]:
#90 dan buyuk, 180 den kucuk degerlerin ortalamasi
df.loc[(df['days']>90)& (df['days']<=180),'Rating'].mean()

np.float64(4.754297994269341)

In [18]:
#180 den buyuk degerlerin ortalamasi
df.loc[(df['days']>180),'Rating'].mean()

np.float64(4.766461958375981)

In [19]:
# agirlik ortalamasi
df.loc[df['days']<=30,'Rating'].mean()*28/100 + \
df.loc[(df['days']>30) & (df['days']<=90),'Rating'].mean()*26/100+\
df.loc[(df['days']>90)& (df['days']<=180),'Rating'].mean()*24/100+\
df.loc[(df['days']>180),'Rating'].mean()*22/100

np.float64(4.764323219188473)

In [20]:
#fonksiyonlastirma
def time_based_weighted_average(dataframe,w1=28,w2=26,w3=24,w4=22):
  return dataframe.loc[df['days']<=30,'Rating'].mean() * w1 /100 + \
  dataframe.loc[(df['days']>30) & (df['days']<=90),'Rating'].mean() * w2 /100 + \
  dataframe.loc[(df['days']>90)& (df['days']<=180),'Rating'].mean() * w3 /100 +\
dataframe.loc[(df['days']>180),'Rating'].mean() * w4 /100

time_based_weighted_average(df)

np.float64(4.764323219188473)

In [21]:
time_based_weighted_average(df,30,25,22,27)

np.float64(4.955409585309535)

# User - Based Weighted Average

In [22]:
df.groupby('Progress').agg({'Rating':'mean'})

,Rating
Progress,
0.00000,4.67391
1.00000,4.64269
2.00000,4.65476
3.00000,4.66355
4.00000,4.77733
5.00000,4.69821
6.00000,4.75510
7.00000,4.73256
8.00000,4.74194


In [23]:
df.loc[df['Progress']<=10,'Rating'].mean()* 22 / 100 + \
df.loc[(df['Progress']<10)&(df['Progress']<=45),'Rating'].mean() * 24 /100 +\
df.loc[(df['Progress']>45)&(df['Progress']<=75),'Rating'].mean() * 26 / 100 + \
df.loc[df['Progress']>75,'Rating'].mean() * 28 / 100

np.float64(4.779381766790847)

In [25]:
# fonksiyonlastirma
def user_based_weighted_average(dataframe,w1=22,w2=24,w3=26,w4=28):
  return dataframe.loc[dataframe['Progress']<=10,'Rating'].mean()* w1 / 100 + \
  dataframe.loc[(dataframe['Progress']<10)&(dataframe['Progress']<=45),'Rating'].mean() * w2 /100 +\
  dataframe.loc[(dataframe['Progress']>45)&(dataframe['Progress']<=75),'Rating'].mean() * w3 / 100 + \
  dataframe.loc[dataframe['Progress']>75,'Rating'].mean() * w4 / 100

user_based_weighted_average(df)

np.float64(4.779381766790847)

# Weighted Rating

In [26]:
def course_weighted_rating(dataframe,time_w=50,user_w=50):
  return time_based_weighted_average(dataframe) * time_w/100 + user_based_weighted_average(dataframe) * user_w/100


course_weighted_rating(df)

np.float64(4.77185249298966)

In [27]:
course_weighted_rating(df,time_w=40,user_w=60)

np.float64(4.773358347749897)